In [8]:
import ee
from Date_format import format_date
from merge_S2_MODIS import merge_images
import pandas as pd
from config import Map
from raster_collection import RasterCollection
from Indices import Sentinel2Indices 
from vis_params import VisualizationParams
from merge_S2_MODIS import merge_images
from meta_data_to_pandas import MetadataExtractor
from export_manager import GEEExportManager, ExportMode
# ee.Authenticate()  # for 1st time you need to authenticate
ee.Initialize()


In [2]:
# Initialize RasterCollection with config and indices
raster_proc = RasterCollection(config=__import__('config'), indices_class=Sentinel2Indices)
# -----------------------------
# Sentinel-2 Processing
# -----------------------------
s2_masked = raster_proc.process_sentinel2()

# visualize map
Map.addLayer(
    s2_masked,
    VisualizationParams.sentinel2_rgb(),
    'Sentinel-2 RGB'
)
Map



Map(center=[44.0125043483778, -120.4227606155169], controls=(WidgetControl(options=['position', 'transparent_b…

In [3]:
# MODIS Processing
# -----------------------------
modis = raster_proc.load_modis()

# visualize modis
Map.addLayer(
    modis.mean(),
    VisualizationParams.modis_albedo_raw(),
    'MODIS Albedo (raw)'
)
Map

Map(bottom=95608.0, center=[44.0125043483778, -120.4227606155169], controls=(WidgetControl(options=['position'…

In [4]:
# join MODIS and S2 on common dates
# Join by date
join = ee.Join.inner()
date_filter = ee.Filter.equals(leftField='date', rightField='date')
joined = join.apply(s2_masked, modis, date_filter)

In [5]:
# Merge MODIS and S2 Collection to single collection
mergedCollection = ee.ImageCollection(joined.map(merge_images))

# Dispaly collection
Map.addLayer(mergedCollection.first(),VisualizationParams.sentinel2_rgb(),"merged")
Map


Map(bottom=95608.0, center=[44.01257086123087, -120.42251586914064], controls=(WidgetControl(options=['positio…

# Export the information about no. of common images (MODIS and S2) as csv
Run this is you would like to export else leave it

In [ ]:
# # Convert to FeatureCollection
# metadata_fc = ee.FeatureCollection(
#     mergedCollection.map(MetadataExtractor.extract_metadata)
# )

# # Get data to client side
# metadata_list = metadata_fc.getInfo()['features']

# # Convert to pandas DataFrame
# image_data = pd.DataFrame([f['properties'] for f in metadata_list])

# # Optional: convert timestamp to datetime
# image_data['system_time_start'] = pd.to_datetime(image_data['system_time_start'], unit='ms', errors='coerce')

# # Save to CSV if needed
# image_data.to_csv("merged_image_metadata.csv", index=False)

# # Print preview
# print(image_data.head())

from metadata_manager import GEECollectionMetadataManager
from meta_data_to_pandas import MetadataExtractor

metadata_manager = GEECollectionMetadataManager(
    collection=mergedCollection,
    extractor=MetadataExtractor.extract_metadata,
    csv_path="merged_image_metadata.csv"
)

image_data = metadata_manager.run()



         date                                                 id  \
0  2023-04-06  20230406T183921_20230406T183958_T11TMG_2023_04_01   
1  2023-04-06  20230406T183921_20230406T183958_T11TMG_2023_04_02   
2  2023-04-06  20230406T183921_20230406T183958_T11TMG_2023_04_03   
3  2023-04-06  20230406T183921_20230406T183958_T11TMG_2023_04_04   
4  2023-04-06  20230406T183921_20230406T183958_T11TMG_2023_04_05   

        system_time_start  
0 2023-04-06 18:52:41.976  
1 2023-04-06 18:52:41.976  
2 2023-04-06 18:52:41.976  
3 2023-04-06 18:52:41.976  
4 2023-04-06 18:52:41.976  


# Batch export the images

In [ ]:

exporter = GEEExportManager(
    collection=mergedCollection,  # exort collect at 500m scale
    export_folder="Exports_2021",
    export_mode=ExportMode.MERGED_MODIS   # use ExportMode.S2_10M  for exporting S2 at 10m resolution, ExportMode.MERGED_MODIS to export merged collection at 500m scale , modis_only
)

exporter.run()




🚀 Export mode: merged_modis_500m
📝 Failure log: export_failures_merged_modis_500m_20251222_231009.csv


🔹 Exporting 1/1856: MERGED_500m_2023-04-06_20230406T183921_20230406T183958_T11TMG_2023_04_01
✔ Started: MERGED_500m_2023-04-06_20230406T183921_20230406T183958_T11TMG_2023_04_01

🔹 Exporting 2/1856: MERGED_500m_2023-04-06_20230406T183921_20230406T183958_T11TMG_2023_04_02
✔ Started: MERGED_500m_2023-04-06_20230406T183921_20230406T183958_T11TMG_2023_04_02

🔹 Exporting 3/1856: MERGED_500m_2023-04-06_20230406T183921_20230406T183958_T11TMG_2023_04_03
✔ Started: MERGED_500m_2023-04-06_20230406T183921_20230406T183958_T11TMG_2023_04_03


KeyboardInterrupt: 